In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install sentence_transformers==0.3.9

Looking in indexes: http://pypi.jooble.com:8080
     |████████████████████████████████| 64 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 32.4 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-py3-none-any.whl size=101032 sha256=b0deddd8a8b5ae5a1aa405b7f654a853ed0992baa8e4dbf533fef1f5a515f9db
  Stored in directory: /Users/ntr/Library/Caches/pip/wheels/a2/37/5a/dc15355b55ec28d07d59ef19f8240fde65bb654d05233205da
Successfully built sentence-transformers
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.95
    Uninstalling sentencepiece-0.1.95:
      Successfully uninstalled sentencepiece-0.1.95
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tok

In [3]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, LabelAccuracyEvaluator
from sentence_transformers.readers import *
import logging
from datetime import datetime
import gzip
import csv
import os

from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import csv
import sys

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

def _read_tsv(input_file, quotechar=None):
    """Reads a tab separated value file."""
    with open(input_file, "r", encoding='utf-8') as f:
        reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
        lines = []
        for line in reader:
            if sys.version_info[0] == 2:
                line = list(unicode(cell, 'utf-8') for cell in line)
            lines.append(line)
        return lines
    
def _create_examples_snli(lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):
        if i == 0:
            continue
        guid = "%s-%s" % (set_type, line[0])
        text_a = line[7]
        text_b = line[8]
        label = line[-1]
        examples.append([guid, text_a, text_b, label])
    return examples

def _create_examples_mnli(lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):
        if i == 0:
            continue
        guid = "%s-%s" % (set_type, line[0])
        text_a = line[8]
        text_b = line[9]
        label = line[-1]
        examples.append([guid, text_a, text_b, label])
    return examples

train_snli = _create_examples_snli(_read_tsv('/kaggle/input/general-language-understanding-evaluation/SNLI/SNLI/train.tsv'), 'train_s')
dev_snli = _create_examples_snli(_read_tsv('/kaggle/input/general-language-understanding-evaluation/SNLI/SNLI/dev.tsv'), 'dev_s')
test_snli = _create_examples_snli(_read_tsv('/kaggle/input/general-language-understanding-evaluation/SNLI/SNLI/test.tsv'), 'test_s')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/general-language-understanding-evaluation/SNLI/SNLI/train.tsv'

In [5]:
# Convert the dataset to a DataLoader ready for training
logging.info("Read AllNLI train dataset")
label2int = {"contradiction": 0, "entailment": 1, "neutral": 2}
train_nli_samples = []
dev_nli_samples = []
test_nli_samples = []

for row in tqdm(train_snli):
    label_id = label2int[row[3]]
    train_nli_samples.append(InputExample(guid = row[0], texts=[row[1], row[2]], label=label_id))
for row in tqdm(dev_snli):
    label_id = label2int[row[3]]
    dev_nli_samples.append(InputExample(guid = row[0], texts=[row[1], row[2]], label=label_id))
for row in tqdm(test_snli):
    label_id = label2int[row[3]]
    test_nli_samples.append(InputExample(guid = row[0], texts=[row[1], row[2]], label=label_id))

  0%|          | 0/549367 [00:00<?, ?it/s]

  0%|          | 0/9842 [00:00<?, ?it/s]

  0%|          | 0/9824 [00:00<?, ?it/s]

In [4]:
# Read the dataset
model_name = 'bert-base-uncased'
batch_size = 64

# Use BERT for mapping tokens to embeddings
word_embedding_model = models.BERT(model_name)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_data_nli = SentencesDataset(train_nli_samples, model=model)
train_dataloader_nli = DataLoader(train_data_nli, shuffle=True, batch_size=batch_size)
train_loss_nli = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=len(label2int))


dev_data_nli = SentencesDataset(dev_nli_samples, model=model)
dev_dataloader_nli = DataLoader(dev_data_nli, shuffle=True, batch_size=batch_size)

test_data_nli = SentencesDataset(test_nli_samples, model=model)
test_dataloader_nli = DataLoader(test_data_nli, shuffle=True, batch_size=batch_size)

2021-02-23 19:23:41 - Lock 140447044895648 acquired on /Users/ntr/.cache/torch/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock



2021-02-23 19:23:42 - Lock 140447044895648 released on /Users/ntr/.cache/torch/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock
2021-02-23 19:23:42 - Lock 140447200334704 acquired on /Users/ntr/.cache/torch/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock



2021-02-23 19:23:54 - Lock 140447200334704 released on /Users/ntr/.cache/torch/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock
2021-02-23 19:23:57 - Lock 140446517651632 acquired on /Users/ntr/.cache/torch/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock



2021-02-23 19:23:58 - Lock 140446517651632 released on /Users/ntr/.cache/torch/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
2021-02-23 19:23:58 - Use pytorch device: cpu


NameError: name 'train_nli_samples' is not defined

In [15]:
# # Read the dataset and initial model
# model_name = 'bert-base-uncased'
# model_save_path = 'output/snli_'+model_name+'-full_iteration_'

# num_epochs = 1
# warmup_steps = math.ceil(len(train_data_nli) * num_epochs / batch_size * 0.1) #10% of train data for warm-up
# logging.info("Warmup-steps: {}".format(warmup_steps))
# train_objectives = [(train_dataloader_nli, train_loss_nli)]

# validation_performance = []
# test_performance = []

# test_evaluator = LabelAccuracyEvaluator(test_dataloader_nli, name = 'nli_test', softmax_model = train_loss_nli)
# dev_evaluator = LabelAccuracyEvaluator(dev_dataloader_nli, name = 'nli_test', softmax_model = train_loss_nli)

# validation_performance.append(model.evaluate(dev_evaluator))
# test_performance.append(model.evaluate(test_evaluator))
# print(f'Iteration - {0} ...')
# print(f'Validation performance - {validation_performance[-1]} ...')
# print(f'Test performance - {test_performance[-1]} ...')

for i in range(1):
    model.fit(train_objectives=train_objectives, output_path=model_save_path+str(i+1))
#     validation_performance.append(model.evaluate(dev_evaluator))
#     test_performance.append(model.evaluate(test_evaluator))
#     print(f'Iteration - {i+1} ...')
#     print(f'Validation performance - {validation_performance[-1]} ...')
#     print(f'Test performance - {test_performance[-1]} ...')
#     model.save("kaggle/working/" + model_save_path+str(i+1))

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8584 [00:00<?, ?it/s]

In [16]:
model.save("bert_model_trained")

In [17]:
!zip -vr results_2.zip bert_model_trained/

  adding: bert_model_trained/	(in=0) (out=0) (stored 0%)
  adding: bert_model_trained/config.json	(in=28) (out=28) (stored 0%)
  adding: bert_model_trained/1_Pooling/	(in=0) (out=0) (stored 0%)
  adding: bert_model_trained/1_Pooling/config.json	(in=190) (out=100) (deflated 47%)
  adding: bert_model_trained/modules.json	(in=228) (out=111) (deflated 51%)
  adding: bert_model_trained/0_BERT/	(in=0) (out=0) (stored 0%)
  adding: bert_model_trained/0_BERT/config.json	(in=502) (out=272) (deflated 46%)
  adding: bert_model_trained/0_BERT/pytorch_model.bin .........................................	(in=438015479) (out=405261278) (deflated 7%)
  adding: bert_model_trained/0_BERT/vocab.txt 	(in=231508) (out=109776) (deflated 53%)
  adding: bert_model_trained/0_BERT/special_tokens_map.json	(in=112) (out=67) (deflated 40%)
  adding: bert_model_trained/0_BERT/sentence_bert_config.json	(in=27) (out=27) (stored 0%)
  adding: bert_model_trained/0_BERT/tokenizer_config.json	(in=300) (out=180) (deflated 

<a href="Your file path"> Download File </a>

In [18]:
import torch
torch.save(train_loss_nli.classifier.cpu(), 'classifier_model_2')

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-uncased')

Exception when trying to download https://sbert.net/models/bert-base-uncased.zip. Response 404


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [7]:
texts = [i[1] for i in train_snli] + [i[2] for i in train_snli] \
        + [i[1] for i in test_snli] + [i[2] for i in test_snli] \
        + [i[1] for i in dev_snli] + [i[2] for i in dev_snli]

In [19]:
texts = list(set(texts))
embeds = model.encode(texts, convert_to_numpy=False)

Batches:   0%|          | 0/20360 [00:00<?, ?it/s]

In [20]:
embeds_cpu = [i.cpu() for i in embeds]

In [23]:
cache = {t:e for t,e in zip(texts, embeds_cpu)}

In [26]:
import pickle

with open('texts_cache.pickle', 'wb') as handle:
    pickle.dump(cache, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [27]:
print("done")

done


In [1]:
cache

NameError: name 'cache' is not defined

In [28]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink(r'texts_cache.pickle')

SyntaxError: invalid syntax (<ipython-input-28-5b387e5c1cb3>, line 2)

In [10]:
bert_model_path = "../modules/models/bert_model_trained_2/"
classification_model_path = "../modules/models/classifier_model_2"

In [12]:
import torch
bert_model = SentenceTransformer(bert_model_path)
classification_model = torch.load(classification_model_path)

2021-02-23 19:26:25 - Load pretrained SentenceTransformer: ../modules/models/bert_model_trained_2/
2021-02-23 19:26:25 - Load SentenceTransformer from folder: ../modules/models/bert_model_trained_2/
2021-02-23 19:26:27 - Use pytorch device: cpu


In [13]:
classification_model

Linear(in_features=2304, out_features=3, bias=True)